In [13]:
import os
cmd = 'pip install tensorflow --ignore-installed'
os.system(cmd)

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 585.9/585.9 MB 513.3 kB/s eta 0:00:00
  Using cached tensorflow_estimator-2.12.0-py2.py3-none-any.whl (440 kB)
  Using cached libclang-16.0.0-py2.py3-none-manylinux2010_x86_64.whl (22.9 MB)
  Using cached opt_einsum-3.3.0-py3-none-any.whl (65 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 6.4 MB/s eta 0:00:00
  Using cached jax-0.4.10.tar.gz (1.3 MB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
  Using cached termcolor-2.3.0-py3-none-any.whl (6.9 kB)
  Using cached tensorboard-2.12.3-py3-none-any.whl (5.6 MB)
  Us

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
mysql-connector-python 8.0.31 requires protobuf<=3.20.1,>=3.11.0, but you have protobuf 4.23.0 which is incompatible.


0

In [16]:
cmd = 'pip install opencv-python --ignore-installed'
os.system(cmd)

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.8/61.8 MB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 6.0 MB/s eta 0:00:00


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.12.0 requires numpy<1.24,>=1.22, but you have numpy 1.24.3 which is incompatible.


0

In [5]:
# importing libraries

import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import os
# import seaborn as sns
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import plot_model
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Dense, Flatten, ZeroPadding2D, GlobalAveragePooling2D

2023-05-12 13:16:30.675738: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-12 13:16:31.324897: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-05-12 13:16:31.324977: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-05-12 13:16:33.155687: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

In [6]:
# Hyper parameters
BATCH_SIZE = 32
IMG_WIDTH, IMG_HEIGHT = 64, 64

TRAIN_DATA_PATH = './split_mel_test_train/train/'
TEST_DATA_PATH = './split_mel_test_train/test'
VALID_DATA_PATH = './split_mel_test_train/val'


In [7]:
def get_generator_from_dir(directory_path, batch_size_):
    data_gen = ImageDataGenerator(rescale=1./255)
    data_batch_generator = data_gen.flow_from_directory(
        directory_path, color_mode = 'rgb', target_size = (IMG_WIDTH, IMG_HEIGHT), 
        batch_size=batch_size_)
    
    return data_batch_generator

In [8]:
train_data = get_generator_from_dir(TRAIN_DATA_PATH, BATCH_SIZE)
validation_data = get_generator_from_dir(VALID_DATA_PATH, BATCH_SIZE)
test_data = get_generator_from_dir(TEST_DATA_PATH, BATCH_SIZE)

Found 1686 images belonging to 3 classes.
Found 210 images belonging to 3 classes.
Found 214 images belonging to 3 classes.


In [13]:
activation_layer = tf.keras.layers.LeakyReLU(alpha=0.1)
visible = Input(shape=(IMG_WIDTH,IMG_HEIGHT,3),name='ip')
conv1 = Conv2D(128, kernel_size=3, strides=(1,1), padding="valid", activation=activation_layer,name='cn1')(visible)
pool1 = MaxPooling2D(pool_size=(3, 3),strides=(2,2), padding="valid",name='pl1')(conv1)

conv2 = Conv2D(64, kernel_size=5, strides=(2,2), padding="valid", activation=activation_layer,name='cn2')(pool1)
pool2 = MaxPooling2D(pool_size=(3, 3),strides=(2,2), padding="valid",name='pl2')(conv2)

flat = Flatten(name='flat')(pool2)
hidden1 = Dense(32, activation=activation_layer,name='hd1')(flat)
output = Dense(3, activation='softmax',name='op')(hidden1)
model = Model(inputs=visible, outputs=output)

# Optimizers, loss function and performance metrics
opt = tf.keras.optimizers.Adam(learning_rate=1e-4)
cat_loss = tf.keras.losses.CategoricalCrossentropy()
acc_metrics = [tf.keras.metrics.CategoricalAccuracy()]

# Compiling the model
model.compile(optimizer=opt, loss=cat_loss, metrics=acc_metrics)

In [14]:
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 ip (InputLayer)             [(None, 64, 64, 3)]       0         
                                                                 
 cn1 (Conv2D)                (None, 62, 62, 128)       3584      
                                                                 
 pl1 (MaxPooling2D)          (None, 30, 30, 128)       0         
                                                                 
 cn2 (Conv2D)                (None, 13, 13, 64)        204864    
                                                                 
 pl2 (MaxPooling2D)          (None, 6, 6, 64)          0         
                                                                 
 flat (Flatten)              (None, 2304)              0         
                                                                 
 hd1 (Dense)                 (None, 32)                7376

In [15]:
model_train = model.fit(train_data, batch_size = BATCH_SIZE, validation_data = validation_data, epochs=10)

Epoch 1/10


2023-05-12 13:27:12.245026: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 62980096 exceeds 10% of free system memory.
2023-05-12 13:27:12.326439: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 62980096 exceeds 10% of free system memory.
2023-05-12 13:27:12.353234: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 14745600 exceeds 10% of free system memory.
2023-05-12 13:27:12.415579: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 14745600 exceeds 10% of free system memory.
2023-05-12 13:27:12.415625: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 23795200 exceeds 10% of free system memory.


53/53 [==============================] - 25s 437ms/step - loss: 0.7197 - categorical_accuracy: 0.6714 - val_loss: 0.5698 - val_categorical_accuracy: 0.6762
Epoch 2/10
53/53 [==============================] - 21s 406ms/step - loss: 0.4960 - categorical_accuracy: 0.7722 - val_loss: 0.4150 - val_categorical_accuracy: 0.8143
Epoch 3/10
53/53 [==============================] - 20s 368ms/step - loss: 0.3641 - categorical_accuracy: 0.8547 - val_loss: 0.3339 - val_categorical_accuracy: 0.8571
Epoch 4/10
53/53 [==============================] - 19s 360ms/step - loss: 0.2869 - categorical_accuracy: 0.8808 - val_loss: 0.2525 - val_categorical_accuracy: 0.9095
Epoch 5/10
53/53 [==============================] - 20s 376ms/step - loss: 0.2277 - categorical_accuracy: 0.9122 - val_loss: 0.2064 - val_categorical_accuracy: 0.9333
Epoch 6/10
53/53 [==============================] - 19s 359ms/step - loss: 0.1770 - categorical_accuracy: 0.9389 - val_loss: 0.1775 - val_categorical_accuracy: 0.9143
Epoch 7/1

In [16]:
y = model.evaluate(test_data)

7/7 [==============================] - 1s 107ms/step - loss: 0.0884 - categorical_accuracy: 0.9673


In [21]:
model.save('cdd_model_2.h5')

In [17]:
train_data.class_indices

{'Distress': 0, 'Egg': 1, 'Feed': 2}

In [24]:
img = tf.keras.preprocessing.image.load_img('split_mel_test_train/test/Distress/distress2-07.jpg', target_size=(IMG_WIDTH, IMG_HEIGHT))
x = tf.keras.preprocessing.image.img_to_array(img)
x = np.expand_dims(x, axis=0)

images = np.vstack([x])
classes = model.predict(images, batch_size=BATCH_SIZE)
classes

1/1 [==============================] - 0s 187ms/step


array([[1., 0., 0.]], dtype=float32)